# Introduction

This notebook serves as an exploration of orbital debris modeling and an introduction to ODAP (Orbital Debris Analysis with Python). While ODAP is currently being refined and converted into a standalone module, you will be able to explore how the module functions and can be utilized in research throughout this notebook.

Initially, this project started from a personal curiosity about how orbital debris works, but later was the basis for a senior thesis written while attending the Harriet L. Wilkes Honors College. The purpose is to develop a modern open-source python implementation of the NASA Standard Breakup Model that others can use to research orbital debris.

Since this project is on going, please note that some functionality may not be working as expected as I continue to go through the process of optimizing and validating the implementations of the various components.

# Table of Contents
### 0. [Packages](#packages)
Covers the initial setup to enable the notebook to function correctly
### 1. [Data Source](#data-source)
Loading real world data from Two Line Elements to use as the foundation for the rest of the simulations
### 2. [Fragmentation Event Modeling](#fragmentation)
### 3. [Cloud Formation and Propagation](#cloud)
### 4. [Analysis](#second-bullet)


<a class="anchor" id="packages"></a>
<h1>0. Packages</h1>

For the purposes of this notebook I will be using a variety of other common modules such as NuMpy, pandas, and Plotly. As such, the below cell will import all necessary modules, as well as import various components from ODAP that will be useful.

In [1]:
# System lib.
import sys
import os
from itertools import product, combinations
from enum import IntEnum
from importlib import reload

# 3rd party lib.
import numpy as np
import re
import skyfield.sgp4lib as spg4
import matplotlib 
import PyQt5
import chart_studio.plotly as py
import chart_studio
import plotly.graph_objects as go
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import datetime as datetime
from skyfield.api import wgs84
from enum import IntEnum    
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from scipy import integrate
from numba import njit, prange

# User defined lib.
if not os.path.join(sys.path[0], '..') in sys.path:
    sys.path.insert(1, os.path.join(sys.path[0], '..'))

import odap.generate_debris as gd
import odap.CoordTransforms as ct
import odap.visualize_deb as vis
import data.planetary_data

matplotlib.use('Agg')
%matplotlib qt5
debris_category = IntEnum('Category', 'rb sc soc')

<IPython.core.display.Javascript object>

To streamline the process of visualizations, I make frequently save various plots to a folder. Additionally, I have critical plots uploaded to Plotlys cloud service for easy sharing. This cell handles both creating the folder structure and authenticating with Plotly's server.

In [ ]:
# Create Directory to Save Figures to
if not os.path.exists("figures"):
    os.mkdir("figures")

# Create Directory to Save Interactive plots to
if not os.path.exists("plots"):
    os.mkdir("plots")
    
# Create Directory to Save gifs
if not os.path.exists("gifs"):
    os.mkdir("gifs")
    
# Retreiving API Keys from OS
PLOTLY_API_KEY = os.environ.get('PLOTLY_API_KEY')
PLOTYLY_USERNAME = 'rhumphreys2017'

# Log into chart studio for uploading plots
chart_studio.tools.set_credentials_file(username=PLOTYLY_USERNAME, api_key=PLOTLY_API_KEY)

<a class="anchor" id="data-source"></a>
<h1>1. Data Source</h1>

To perform accurate simulations, it is essential to have some real-world satellite data as the starting point for modeling the fragmentation event. Therefore, this notebook is a file containing NORAD Two-Line Element Sets (TLE) acquired from [CelesTrak](https://celestrak.com/NORAD/elements/). The rest of this section involves importing said data and discussing how the TLE data structure functions.

<h3>1.1 Loading TLE Data</h3>

TLE's are standardized data structures that contain the orbital elements used to describe Earth-orbiting objects for a given point in time. Most importantly, they are used to determine where a given object will be at any given time. Thus, it is a valuable tool for analyzing potential orbital collisions as well as tracking orbital debris.

To acquire the most recent information about all objects being tracked in Earth orbit is recommended to download the latest TLE data from CelesTrak. This file comes in the form of a `.txt` file that first must be parsed to use while programming. While it is possible to manually parse all of the data from the TLEs using [Regular expression operations](https://docs.python.org/3/library/re.html), for this notebook, I will be utilizing the `skyfield` python module as it has the built-in functionality to handle this.

In [2]:
# Opening the .txt file
import importlib
import sys
from odap.Satellite import Satellite
from odap.TLE import TLE

with open(sys.path[1] + "/data/3le.txt") as f:
    txt = f.read()
    
# Using regular expression to perform basic parsing of the 3le.txt file
# Returns and array of arrays where each subarray contains three strings
# corresponding to each line of th TLE
tles_data = np.array([tle for tle in re.findall('(.*?)\n(.*?)\n(.*?)\n', txt)])

# Convert name line of TLE to correct format
tles_data[:, 0] = [name[2: ] for name in tles_data[:, 0]] 

tles = [TLE(lines[0], lines[1], lines[2]) for lines in tles_data]
tles

 ...]

<h3>1.2 Select orbital object for analysis</h3>

Now that we have an array containing Satellite objects, we can utilize NumPy to find a satellite that satisfies whatever criterion we are looking for. For example, we may want to simulate a fragmentation event for a satellite with a low semi-major axis or a high eccentricity. To keep things simple, I selected a satellite using the name assigned to it by NORAD.

In [5]:
# Constructing an array that contains the names of all the satellites in our dataset
# Note: The name formatting of all satellites starts with "0 ",
#       thus we slice the String to cut the 0.
desigs = np.array([tle.int_desig for tle in tles])

# Search the name array for the index of desired satellite
i = np.argwhere(desigs == "99066B").flatten()[0]

# Retrieve that satellite object from the `satellites` array using the found index
tle = tles[i]

# satellite.compute_characteristic_length()
# satellite.get_orbital_elements()

# apoapsis = satellite.elements.a * (1 + float(satellite.elements.eccentricity)) / 1e3
# periapsis = satellite.elements.a * (1 - float(satellite.elements.eccentricity)) / 1e3

SyntaxError: invalid syntax (993132939.py, line 10)

Now that we have a specific satellite object, we can access all of the relevant information easily by utilizing the attributes specified by the `Satellite` module.

In [ ]:
# Displaying the TLE data for the retrieved satellite
print("Name: " + satellite.name)
print("Mean Anomaly: ", satellite.mean_anomaly)
print("BSTAR drag: ", satellite.bstar)

<a class="anchor" id="fragmentation"></a>
# 2. Fragmentation Event Modeling

A satellite breakup model is a mathematical model used to describe the outcome of a satellite breakup due to an explosion or collision. A satellite breakup model should describe the size, area-to-mass (AM) ratio, and the ejection velocity of each fragment produced in the satellite breakup. The most easily accessible literature model is the [NASA Standard breakup model](https://www.sciencedirect.com/science/article/abs/pii/S0273117701004239). This model is implemented in ODAP in `generate_debris.py`.

In the following subsections, ODAP is utilized to simulate an explosion event and a collision event. Additionally, some information about how the NASA Standard Breakup Model works is provided. For additional information it is recomended to refer to ...

## 2.1 Exploring the NASA Breakup model

The NASA standard breakup model uses experimental observations performed both on Earth and in orbit to characterize the breakup using statistical distributions. The choice to use statistical distributions is a result of the stochastic nature of the breakup event, meaning it would be impossible to reproduce the same circumstances each time. By using a statistical distribution and sampling from it we can more accurately represent the fragments that would be generated during a collision or explosion.

### Characteristic Length

To account for the different characteristics of each fragment of debris, the statistical distributions must be expressed as a function of some independent variable. In the latest version of the NASA breakup model, this variable is called the characteristic length, denoted $L_c$. By defining the distributions using characteristic length we ensure that the mass, area, and velocity of each fragment are not constant for all debris with the same characteristic length. The implementation of the characteristic length distribution can be cumbersome to follow. As such, a flow chart illustrating the steps of the algorithm is provided below.
<br>
<img src="../images/diagram1.png" width=400 height=400 />

### Area to Mass Distribution

The area-to-mass ratio, A/M , for fragments is a distribution that was based on analysis of thousands of fragmentation debris and provides us with a method to determine the mass of each fragment of debris. The discrete distributions were found by using a $\chi^2$ fit to orbital decay characteristics for 1,780 upper stage explosion fragments, and similar data was developed for spacecraft fragments. Each type of debris producer - rocket bodies (RB), spacecraft (SC), satellites (SAT) - will produce different size debris. As such, the distribution that determines the area to mass ratio has three variants. All three are based on a normal distribution but use different expressions for determining the mean and standard deviation of the distribution.

For example, small objects with $L_c < 8$cm, SAT, the $A/M$ distribution is expressed as

\begin{equation}
	D_{A/M}(\lambda_c, \chi) = \mathcal{N}(\mu_{A/M}(\lambda_c), \sigma_{A/M}(\lambda_c), \chi).
\end{equation}

$D_{A/M}$ is the distribution function of $\chi$ as a function of $\lambda_c$, where
\begin{align}
	\lambda_c &= \log_{10}(L_c),\\
	\chi &= \log_{10}(A/M)
\end{align}

$\mathcal{N}$ is the normal distribution function with mean $\mu_{A/M}$ and standard deviation $\sigma_{A/M}$, where

\begin{align}
	 \mu_{A/M} &= \begin{cases} 
		-0.3, & \lambda_c\leq -1.75 \\
		-0.3 - 1.4(\lambda_c + 1.75), & -1.75 < \lambda_c <-1.25 \\
		-1.0, & \lambda_c \geq -1.25 
	\end{cases}\\
	\text{and}\\
	\sigma_{A/M} &= \begin{cases} 
		0.2, & \lambda_c \leq -3.5 \\
		0.2 + 0.1333(\lambda_c + 3.5) & \lambda_c > -3.5 \\
	\end{cases}
\end{align}

Every fragment of debris has a corresponding $A/M$ distribution since both $\mu_{A/M}$ and $\sigma_{A/M}$ are functions of $\lambda_c$. To determine the corresponding $A/M$ ratio for each debris, a random value is drawn from the distribution. This accounts for stochastic nature of breakup events mentioned previously. 


The $A/M$ ratio alone does not provide enough information to determine both the area and mass of a fragment. As such, the average cross-sectional area, $A$, can also be obtained through a one-to-one correspondence with $L_c$ using the following expression:

\begin{align}
	A_x = \begin{cases}
		0.540424 * L_c^2 & \text{where } L_c < 0.00167 \text{ m} \\
		0.556945 * L_c^{2.0047077} & \text{where } L_c \geq 0.00167 \text{ m} \\
	\end{cases}
\end{align}

Utilizing both the $A/M$ ratio and the cross sectional area $A$, we can now obtain the mass $M$ easily using
$M = A_x / (A/M)$.

### Change in Velocity Distribution

The differential amount of velocity that each fragment will gain due to the breakup event is determined in a similar manner to the $A/M$ ratio. The notable differences are that the distribution is now a log-normal distribution and that an additional check is implemented to ensure that extremely high ejection velocities are not included in the case of collisions. 

More explicitly, the velocity check is performed by sampling a value from the velocity distribution and checking if it is lower than $1.3v_c$, where $v_c$ is the relative collision velocity. If the value fails the check then new values are drawn until the check is passed.

The change in velocity, $\Delta v$, is modeled by log-normal distribution that is function of the $A/M$ ratio by

\begin{equation}
	D_{\Delta v}= \mathcal{N}(\mu_{v}(\chi), \sigma_{v}(\chi), \xi)
\end{equation}
where for SAT breakups \begin{align}
	&\xi = \log_{10}(\Delta v),\\
	&\chi = \log_{10}(A/M),\\
	&\mu_{v}(\chi) = 0.2\chi+1.85,\\
	&\sigma_{v}(\chi) = 0.4,
\end{align}

### Additional information

All of the functionality required for modeling RB, SC, and SAT fragmentation events is included in ODAP. Speciffically, the implementation details can be found in `generate_debris.py`. It is recomended that for additional information you consult [my thesis](), [letizia](), [another](), [another]().


### 2.1.1 Modeling a rocket body explosion

Now that the details of the NASA Breakup Model have been specified, we can begin to simulate some fragmentation events. This first example shows how to simulate an explosion of a 1000 kg rocket body. An outputs some information about the debris cloud generated.

In [ ]:
### Testing Fragmentation Event
import odap.FragmentationEvent as event
import odap.SimulationConfiguration as configuration

reload(configuration)
reload(event)


config = configuration.SimulationConfiguration('../data/testData.ini')
event = event.FragmentationEvent(config, satellite)
event.run()

In [ ]:
#
# Explosion: 
#
# Simulating an explosion of a 1000 kg rocket body. Note that since the object is
# exploding there is no projectile thus `m_projectile` is 0 and `v_impact` is 0.


m_target        = 1000      # [kg]. The mass of the target
m_projectile    = 0         # [kg]. The mass of the projectile
v_impact        = 0         # [km s^-1]. The relative impact velocity
is_catastrophic = True 
is_explosion    = True
object_type     = debris_category.rb

# `L_c` [m], `areas` [m^2], `masses` [kg], `AM` []
L_c, areas, masses, AM = gd.fragmentation(m_target,
                                          m_projectile,
                                          v_impact,
                                          is_catastrophic,
                                          object_type,
                                          is_explosion)

# Constructing bins from 1e-3 [m] to 1 [m] and populating histogram
# with generated characteristic length data
bins = [1e-3, 1e-2, 1e-1, 1, np.inf]
h,b = np.histogram(L_c, bins=bins)
ch = [ { f'>{bins[i]}m':np.sum(h[i:]) for i in range(len(h))}]
df = pd.DataFrame(ch)

# Constructing bins from 1e3 [g] to inf [g] and populating histogram
# with generated mass data
bins = [1, np.inf]
h,b = np.histogram(masses*1e3, bins=bins) # grams
ch = [ { f'>{bins[i]}g':np.sum(h[i:]) for i in range(len(h))}]
df2 = pd.DataFrame(ch)

# Constructing bins from 1e4 [cm^2] to inf [cm^2] and populating histogram
# with generated mass data
h,b = np.histogram(areas*1e4, bins=bins) # cm^2
ch = [ { f'>{bins[i]}cm^2':np.sum(h[i:]) for i in range(len(h))}]
df3 = pd.DataFrame(ch)

# # # VELOCITY DATA (A bit janky currently, always need to specify v_c event for explosions)
deltaV = 10**np.array(gd.distribution_deltaV(AM, 5, True)) # [m·s^-1]
bins = [100, np.inf]
h,b = np.histogram(deltaV, bins=bins)
ch = [ { f'>{bins[i]}km/s':np.sum(h[i:]) for i in range(len(h))}]
df4 = pd.DataFrame(ch)

# Showing Distribution results for explosion event
results = pd.concat([df, df2, df3, df4], axis=1)
results

### 2.1.1 Modeling a rocket body collision

Additionally, we can also use ODAP to simulate collision events. The only difference is specifying a non zero projectile mass and a non zero relative impact velocity.

In [ ]:
#
# Collision: 
#
# Simulating an collision between a 1000 kg rocket body and a 10 kg projectile. 

gd = reload(gd)

m_target        = 1000       # [kg]
m_projectile    = 10         # [kg]
v_impact        = 10         # [km s^-1]
is_catastrophic = True 
is_explosion    = False
object_type     = debris_category.rb

# `L_c` [m], `areas` [m^2], `masses` [kg], `AM` []
L_c, areas, masses, AM = gd.fragmentation(m_target,
                                          m_projectile,
                                          v_impact,
                                          is_catastrophic,
                                          object_type,
                                          is_explosion)

# Constructing bins from 1e-3 [m] to 1 [m] and populating histogram
# with generated characteristic length data
bins = [1e-3, 1e-2, 1e-1, 1, np.inf] 
h,b = np.histogram(L_c, bins=bins)
ch = [ { f'>{bins[i]}m':np.sum(h[i:]) for i in range(len(h))}]
df = pd.DataFrame(ch)

# Constructing bins from 1e3 [g] to inf [g] and populating histogram
# with generated mass data
bins = [1, np.inf]
h,b = np.histogram(masses*1e3, bins=bins) # grams
ch = [ { f'>{bins[i]}g':np.sum(h[i:]) for i in range(len(h))}]
df2 = pd.DataFrame(ch)

# Constructing bins from 1e4 [cm^2] to inf [cm^2] and populating histogram
# with generated mass data
h,b = np.histogram(areas*1e4, bins=bins) # cm^2
ch = [ { f'>{bins[i]}cm^2':np.sum(h[i:]) for i in range(len(h))}]
df3 = pd.DataFrame(ch)

# # VELOCITY DATA (A bit janky currently, always need to specify v_c event for explosions)
deltaV = 10**np.array(gd.distribution_deltaV(AM, v_impact, is_explosion)) # [m·s^-1]
bins = [1e3, np.inf]
h,b = np.histogram(deltaV, bins=bins)
ch = [ { f'>{bins[i]}m/s':np.sum(h[i:]) for i in range(len(h))}]
df4 = pd.DataFrame(ch)

results = pd.concat([df, df2, df3, df4], axis=1)
results

### 2.1.2 Visualizing the results

Now that we have some data about a breakup event, we can begin to visualize the various variables to understand how closely ODAP models the fragmentation event compared to the results from other available research data.

This is accomplished by utilizing Plotly to generate figures. As such, most of the bellow cell involves setting appropriate layouts to the figures. The primary purpose of which is to illustrate how we can take the fragmentation data and visualize it.

In [ ]:
# Create logarithmic spaced bins
def create_log_bins(values, nbins=100):
    bins = np.geomspace(values.min(), values.max(), nbins)
    a = bins[1]/bins[0]
    bins = np.concatenate([[bins[0]/a], bins,[bins[-1]*a]])
    return bins

# Specify a common Layout theme
layout = dict(
    autosize=False,
    width=500,
    height=500,
    template = 'plotly_white',
    yaxis = dict(
        range=[0,8e5],
        showexponent = 'all',
        exponentformat = 'e'
    ),
    legend=dict(
        y=0.5,
        traceorder='reversed',
        font=dict(
            size=16
        )
    )
)

### Producing Characteristic Length Visual

In [ ]:
# Creating Histogram
h, b = np.histogram(L_c, bins=create_log_bins(L_c))
# Make figure
fig = go.Figure(data=[go.Scatter(x=b, y=h, mode='lines', hoverinfo='all',
                                 line=dict(shape='hvh'))],
                layout=layout)
fig.update_xaxes(type="log")
fig.update_layout(
    title = 'Characteristic Length Distribution',
    xaxis_title=r'$\log_{10}(L_{c}\:[m])$',
    yaxis_title=r'$N_f$'
)

# Save the Plot to the figures folder
fig.write_image("figures/N_f_vs_L_c.png", width=500, height=500, scale=2)
py.iplot(fig, filename="Characteristic Length Distribution")

### Producing Areas Visual

In [ ]:
# Creating Histogram
h, b = np.histogram(areas, bins=create_log_bins(areas))

# Make figure
fig2 = go.Figure(data=[go.Scatter(x=b, y=h, mode='lines', hoverinfo='all',
                                 line=dict(shape='hvh'))],
                layout=layout)
fig2.update_xaxes(type="log")
fig2.update_layout(
    title = 'Area distribution',
    xaxis_title=r'$\log_{10}(A\:[m^2])$',
    yaxis_title=r'$N_f$',
)

# Plot
fig2.write_image("figures/N_f_vs_A.png", width=500, height=500, scale=2)
py.iplot(fig2, filename="Area distribution")

### Producing Mass Visual

In [ ]:
# Creating Histogram
h, b = np.histogram(masses, bins=create_log_bins(masses))

# Make figure
fig3 = go.Figure(data=[go.Scatter(x=b, y=h, mode='lines', hoverinfo='all',
                                 line=dict(shape='hvh'))],
                layout=layout)
fig3.update_xaxes(type="log")
fig3.update_layout(
    title = 'Mass Distribution',
    xaxis_title=r'$\log_{10}(M\:[kg])$',
    yaxis_title=r'$N_f$'
)

# Plot
fig3.write_image("figures/N_f_vs_M.png", width=500, height=500, scale=2)
py.iplot(fig3, filename="Mass Distribution")

### Producing Velocity Visual

In [ ]:
# Creating Histogram
h, b = np.histogram(deltaV, bins=create_log_bins(deltaV))

# Make figure
fig4 = go.Figure(data=[go.Scatter(x=b, y=h, mode='lines', hoverinfo='all',
                                 line=dict(shape='hvh'))],
                layout=layout)

# Update figure
fig4.update_xaxes(type="log")
fig4.update_layout(
    title = 'Velocity distribution',
    xaxis_title=r'$\log_{10}(V\:[ms^-1])$',
    yaxis_title=r'$N_f$',
)

# Plot
fig4.write_image("figures/N_f_vs_V.png", width=500, height=500, scale=2)
py.iplot(fig4, filename="Velocity distribution")

## 2.2 Performing a Fragmentation Event

In [ ]:
# Performing fragmentation event
ts = load.timescale(builtin=True)
t_fragmentation = ts.now()

print(type(ts))
print(type(t_fragmentation))

geocentric      = satellite.at(t_fragmentation)
init_position   = geocentric.position.m

m_target         = 250   # [kg] (Approx. mass of starlink sat)
m_projectile     = 200     # [kg]
v_impact         = 2   # [km·s^-1] (Measured relative to the target) (Needs to be in km·s^-1)
is_catastrophic  = False
is_explosion     = False

L_c, areas, masses, AM = gd.fragmentation(m_target, m_projectile, v_impact, is_catastrophic, debris_category.sc, is_explosion)
deltaV = np.array(gd.distribution_deltaV(AM, v_impact, False)) # Returns as [km·s^-1]
deltaV = deltaV * 1e3    #[m·s^-1]

#### 2.1.3 Retriving Cartesian coordinates of debris fragments

In [ ]:
from numpy.linalg import norm
init_position = geocentric.position.m
deb_positions       = np.empty((len(AM), 3))
deb_positions[:, :] = init_position[None,:]
deb_velocities      = gd.velocity_vectors(len(AM), geocentric.velocity.m_per_s, deltaV)

#### 2.1.4 Converting coordinates to Keplerian elements

In [ ]:
keplerian_state     = ct.rv2coe(deb_positions, deb_velocities, planetary_data.earth['mu'])

# Removing fragments that would renter earth
periapsis           = keplerian_state[0, :] * (1 - keplerian_state[1, :])
I                   = np.argwhere(periapsis > planetary_data.earth['radius'])
keplerian_state     = np.squeeze(keplerian_state[:, I])
areas               = areas[I].flatten()  # When doing the indexing , a 1d dim being added which is unneccesary
masses              = masses[I].flatten() # When doing the indexing , a 1d dim being added which is unneccesary

## 2.2 Ring Formation

In [ ]:
## Shrink number of debris being used
indexes = np.random.default_rng().choice(keplerian_state.shape[1], size=1000, replace=False)

ks = keplerian_state[:, indexes]
masses = masses[indexes]
areas = areas[indexes]
deb_positions = deb_positions[indexes, :]
deb_velocities = deb_velocities[indexes, :]

In [ ]:
import Perturbations as OP
import planetary_data as pd
from Perturbations import null_perts

# Cleanup states to remove any fragments that would deorbit, given no perturbations
periapsis     = ks[0, :] * (1 - ks[1, :])
I             = np.argwhere(periapsis > pd.earth['radius'])
ks_pruned     = np.squeeze(ks[:, I])
T             = ks_pruned[8, :]
areas_pruned  = areas[I].flatten()  # When doing the indexing , a 1d dim being added which is unneccesary
masses_pruned = masses[I].flatten() # When doing the indexing , a 1d dim being added which is unneccesary

# Propagate orbit for a period of time
perts = null_perts()
perts['aero'] = True
perts['J2']   = True
op = OP.OrbitPropagator(ks_pruned, areas_pruned, masses_pruned, [0, 1000*np.ceil(max(T))], 60*30, perts=perts)
op.propagate_orbit()

# Get the cartesian state representation
cartesian_states = op.cartesian_representation()

<h4>3.4.2 Particle Debris flux</h4>

Using a particle flux to determine when the fragments of the debris have finished the formation of the ring. Indicating the end of the first phase of the debris cloud formation. This is accomplished by creating an xz plane and detecting when particles have switched from one side to the other. This approach will cause a peak as fragments pass through that becomes uniform as the debris becomes uniformly spread out.

<h4>3.4.3 Convergence of the flux</h4>

The next step is determining when the fragments have ended the torroid formation phase. This occurs when the fragments are approximately uniformally spread out. We can check to see when the flux meets a convergence criterion to determine when this happens.

Now that the band has formed, we can shift away from propagating the exact position of each fragments and inplace propgate their changes in eccentricity and semi major axis due to drag. To do this first we must get the final states of the debris after the band has formed.

In [ ]:
import time
import matplotlib.dates as mdates
from dateutil import tz

def fragmentation_flux(X):
    return np.sum((X[:-1, :, 1] < 0) & (X[1:, :, 1] > 0), axis=1)
    
position = cartesian_states[:, 0, :, :]
flux = fragmentation_flux(position)

w = 100 # Window of points to look at
tol = 5
convergence_ratio = np.array([np.var(flux[i:i+w])/np.mean(flux[i:i+w]) for i in range(len(flux))])    
intersection_index = np.argwhere(convergence_ratio <= tol).flatten()[0]

# datetimes
t_flux = t_fragmentation.utc_datetime() + np.array(range(len(flux))) * datetime.timedelta(minutes = 5)

# Removing last window from `t_flux`, `flux`, and `convergence_ratio` bc. not well defined for last values
t_flux = t_flux[:-w]
flux = flux[:-w]
convergence_ratio = convergence_ratio[:-w]

# Pruning data to the end of the ring formation
cs_toroid = cartesian_states[:intersection_index, :, :, :]
ks_toroid = op.states[0:intersection_index, :, :]
op.states = ks_toroid

## 2.3 Band Formation

### 2.3.1 Drag Implementation

In [ ]:
import Aerodynamics as aero
import numpy as np


upper_bound = 900                                #[km]
altitudes   = np.arange(0, upper_bound, 1)      #[km]
rho         = aero.atmosphere_density(altitudes) #[kg·m^-3]

I_standard = np.argwhere(altitudes == 25).flatten()[0]
I_cira    = np.argwhere(altitudes  == 500).flatten()[0]

# Plotting the Exponential Atmospheric Model

layout = go.Layout(
    title        = go.layout.Title(text='Altitude (z) vs. Atmospheric Density (ρ)',
                                   x=0.5),
    xaxis_title  = 'z [km]',
    yaxis_title  = '$\log_{10}(\\rho\:[kg·m^{-3}])$',
    template     = 'plotly_white',
    legend       = go.layout.Legend(yanchor="top",
                             y=0.99,
                             xanchor="right",
                             x=0.99)
)

data = [
    go.Scatter(x=altitudes[:I_standard], y=rho[:I_standard],
                    mode='lines',
                    name='U.S Standard Atmosphere'),
    go.Scatter(x=altitudes[I_standard:I_cira], y=rho[I_standard:I_cira],
                    mode='lines',
                    name='CIRA-72'),
    go.Scatter(x=altitudes[I_cira:], y=rho[I_cira:],
                    mode='lines',
                    name='CIRA-72 with T_infinity = 1000K')
]

fig = go.Figure(data=data, layout=layout)
fig.update_yaxes(type="log")


fig.write_image("figures/Atmospheric_Density_v_Altitude.png", width=500, height=500, scale=2)
f2 = go.FigureWidget(fig)
f2

### 2.3.1 Applying Perturbations to Satellite

In [ ]:
from scipy.special import iv
from scipy import integrate
import Aerodynamics as aero

op.tspan[-1] = 3600*24*365*3
op.dt = 3600*24
de, da, di, dOmega, domega, dnu, dp = op.propagate_perturbations()

# 3. Analysis

## 3.1 Flux

### 3.1.1 FLux plot

In [ ]:
# Creating Flux v. Time plot
layout = go.Layout(
    title        = dict(text='$\\text{Flux}\:(\\Phi)\:\\text{vs. Time }(t)$',
                        x=0.5),
    xaxis_title  = '$t\:[days]$',
    yaxis_title  = '$\\text{ Number of fragments passing XZ plane, }\Phi\:$',
    template     = 'plotly_white'
)


data = [
    go.Scatter(x=t_flux, y=flux,
               mode='lines',
               name='Flux'),
    go.Scatter(x=[t_flux[intersection_index], t_flux[intersection_index]], y=[0, np.max(flux)],
               mode='lines',
               line=dict(dash = 'dash'),
               name='Convergence')
]

fig1 = go.Figure(data=data, layout=layout)

# Stopping data to have half before intersection index and half after
index_stop = intersection_index * 2
if index_stop > len(flux) - 1 : index_stop = len(flux) - 1
fig1.update_layout(xaxis_range=[t_flux[0],t_flux[index_stop]])

# Saving plot as an image and uploading it to plotly
fig1.write_image("figures/Flux_v_Time.png", width=500, height=500, scale=2)
#py.iplot(fig1, filename="Flux v. Time")

### 3.1.2 Convergence Ratio plot

In [ ]:
#Creating Convergence Ratio v. Time plot
layout = go.Layout(
    title        = dict(text='Convergence ratio vs. Time (t)',
                        x=0.5),
    xaxis_title  = '$t\:[days]$',
    yaxis_title  = 'Convergence ratio []',
    template     = 'plotly_white',
    legend       = go.layout.Legend(yanchor="top",
                             y=0.99,
                             xanchor="right",
                             x=0.99)
)
data = [
    go.Scatter(x=t_flux, y=convergence_ratio,
               mode='lines',
               name='Convergence ratio'),
    go.Scatter(x=[t_flux[intersection_index], t_flux[intersection_index]], y=[0, np.max(flux)],
               mode='lines',
               line=dict(dash = 'dash'),
               name='Convergence time'),
    go.Scatter(x=[t_flux[0], t_flux[-1]], y=[tol, tol],
               mode='lines',
               line=dict(dash = 'dash'),
               name='Tolerance'),
]
fig2 = go.Figure(data=data, layout=layout)
fig2.update_yaxes(type="log")
fig2.write_image("figures/Convergence_Ratio_v_Time.png", width=500, height=500, scale=2)
#py.iplot(fig2, filename="Convergence Ratio v. Time")

## 3.2 Ring visualization

In [ ]:
import plotly.express as px
import plotly.io as pio
import pandas

spherical_earth_map = np.load('map_sphere.npy') 

pos_toroid = cs_toroid[:, 0, :, :]/1e3
N_timesteps = pos_toroid.shape[0]
N_fragments = pos_toroid.shape[1]
r_E = op.cb['radius'] / 1e3
xm, ym, zm = spherical_earth_map.T * r_E

# Converting data to pandas dataframe
df = pandas.DataFrame()
# *** Update this if chnage timestep in initial orbit propagation ***
dt = 60 * 5 #[s]
# Want to show the evolution in 30 min
timesteps = np.arange(0,N_timesteps, 6)

for t in timesteps:   
    step = t*np.ones_like(N_timesteps)
    time = dt * step / 60 #[min]
    d = {'X': pos_toroid[t, :, 0],
         'Y': pos_toroid[t, :, 1],
         'Z':pos_toroid[t, :, 2],
         'Min.': time,
         'a': ks_toroid[t, 0, :]/1e3,
         'e': ks_toroid[t, 1, :],
         'i': ks_toroid[t, 2, :],
        }
    df = pandas.concat([df, pandas.DataFrame(data=d)])

# Creating visual
def spheres(size, clr, dist=0): 

    # Set up 100 points. First, do angles
    theta = np.linspace(0,2*np.pi,100)
    phi = np.linspace(0,np.pi,100)

    # Set up coordinates for points on the sphere
    x0 = dist + size * np.outer(np.cos(theta),np.sin(phi))
    y0 = size * np.outer(np.sin(theta),np.sin(phi))
    z0 = size * np.outer(np.ones(100),np.cos(phi))

    # Set up trace
    trace= go.Surface(x=x0, y=y0, z=z0, colorscale=[[0,clr], [1,clr]])
    trace.update(showscale=False)

    return trace

fig = px.scatter_3d(
    data_frame=df,
    x='X',
    y='Y',
    z='Z',
    title='Evolution of debris cloud to toroid formation',
    hover_data={'Min.': False, 'X': False, 'Y':False, 'Z':False, 'a':':.1f', 'e':':.4f','i':':.1f' },
    height=800,                 # height of graph in pixels
    width =800,
    animation_frame='Min.',   # assign marks to animation frames
    range_x=[-r_E - 1000,r_E + 1000],
    range_z=[-r_E - 1000,r_E + 1000],
    range_y=[-r_E - 1000,r_E + 1000],

)
fig.update_traces(marker={'size': 3})
# Add Earth
earth=spheres(r_E, '#F0FFFF', 0) # Earth
#fig.add_trace(go.Scatter3d(x=xm, y=ym, z=zm, mode='lines', line=dict(color=zm, colorscale='Viridis')))
fig['layout']['scene']['aspectmode'] = 'cube'
fig.add_trace(earth)
fig.update_layout(transition = {'duration': 2000})
fig.write_html("plots/ring.html")

## 3.3 Band visualization

In [ ]:
temp = np.zeros_like(da) # The params set to 0 dont matter for converting to rv
ks_propagated = np.swapaxes(np.stack([da, de, di, dOmega, domega, temp, dnu, dp, temp, temp]).T, 1, 2)
ks_final = np.concatenate([ks_toroid, ks_propagated])
op.states = ks_final
cs_final = op.cartesian_representation()

In [ ]:
import pandas as pandas
import plotly.express as px

pos_toroid = cs_final[cs_toroid.shape[0]-1:, 0, :, :]/1e3
N_timesteps = pos_toroid.shape[0]
N_fragments = pos_toroid.shape[1]
r_E = op.cb['radius'] / 1e3


# Converting data to pandas dataframe
df = pandas.DataFrame()
# *** Update this if chnage timestep in initial orbit propagation ***
dt = 60 * 5 #[s]
# Want to show the evolution in 1 day steps
timesteps = np.arange(0,N_timesteps, 5)

for t in timesteps:   
    step = t*np.ones_like(N_timesteps)
    time = step  #[day]
    d = {'X': pos_toroid[t, :, 0],
         'Y': pos_toroid[t, :, 1],
         'Z':pos_toroid[t, :, 2],
         'Day': time,
        }
    df = pandas.concat([df, pandas.DataFrame(data=d)])

def spheres(size, clr, dist=0): 

    # Set up 100 points. First, do angles
    theta = np.linspace(0,2*np.pi,100)
    phi = np.linspace(0,np.pi,100)

    # Set up coordinates for points on the sphere
    x0 = dist + size * np.outer(np.cos(theta),np.sin(phi))
    y0 = size * np.outer(np.sin(theta),np.sin(phi))
    z0 = size * np.outer(np.ones(100),np.cos(phi))

    # Set up trace
    trace= go.Surface(x=x0, y=y0, z=z0, colorscale=[[0,clr], [1,clr]])
    trace.update(showscale=False)

    return trace
fig = px.scatter_3d(
    data_frame=df,
    x='X',
    y='Y',
    z='Z',
    title='Evolution of debris cloud to Band formation',
    #labels={'Years in school (avg)': 'Years Women are in School'},
    #hover_data={'Min.': False, 'X': False, 'Y':False, 'Z':False, 'a':':.1f', 'e':':.4f','i':':.1f' },
    #hover_name='Orbital Elements',        # values appear in bold in the hover tooltip
    height=800,                 # height of graph in pixels
    width =800,
    animation_frame='Day',   # assign marks to animation frames
    range_x=[-r_E - 1000,r_E + 1000],
    range_z=[-r_E - 1000,r_E + 1000],
    range_y=[-r_E - 1000,r_E + 1000],

)
fig.update_traces(marker={'size': 1.5, 'color':'#6372f4'})
# Add Earth
earth=spheres(r_E, '#ffffff', 0) # Earth
fig.add_trace(earth)
#fig.add_trace(go.Scatter3d(x=xm, y=ym, z=zm, mode='lines', line=dict(color=zm, colorscale='Viridis')))
fig['layout']['scene']['aspectmode'] = 'cube'
fig.update_layout(transition = {'duration': 2000})
fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)')
fig.write_html("plots/band.html")

## 3.4 Time to deorbit 

In [ ]:
import plotly.figure_factory as ff
import matplotlib.cm as cm

AM = op.A / op.M
z  = (da * (1 - de)) - op.cb['radius']
z[z < 100*1e3] = 0

layout = go.Layout(
    title        = dict(text='Altitude of 50 debris fragments over 3 years',
                        x=0.5),
    xaxis_title  = '$t\:[days]$',
    yaxis_title  = 'Altitude [km]',
    template     = 'plotly_white',
    legend       = go.layout.Legend(yanchor="top",
                             y=0.99,
                             xanchor="right",
                             x=0.99)
)

data = []

for i in range(25):
    alt = np.trim_zeros(z[i, :]) / 1e3
    scatter = go.Scatter(x=[i for i in range(len(alt))], y=alt,
               mode='lines')
    data.append(scatter)
    
fig = go.Figure(data=data, layout=layout)
fig.update_layout(coloraxis=dict(colorscale='RdBu'), showlegend=False)
fig.show()    
fig.write_image("figures/oxp_altitudes.png", width=500, height=500, scale=2)

## 3.5 Debris spread

In [ ]:
index = int(np.ceil(ks_propagated.shape[0]*.10)) # index near begining
raan_0 = ks_propagated[index, 3, :].copy() % 360
raan_0[raan_0 > 180] -= 360 # Converting angles to new range

raan_mid = ks_propagated[ks_propagated.shape[0] // 2, 3, :].copy()  % 360
raan_mid[raan_mid  > 180] -= 360 

raan_f = ks_propagated[-1, 3, :].copy() % 360
raan_f[raan_f > 180] -= 360

In [ ]:
import math
import plotly.figure_factory as ff

uniform_dist = np.random.uniform(-180, 180, len(raan_0))
group_labels = ['$\Omega_{initial}$', '$\Omega_{midpoint}$', '$\Omega_{final}$', 'uniform']
fig = ff.create_distplot([raan_0, raan_mid, raan_f, uniform_dist], group_labels, show_hist =  False)

# Updating the uniform curve to be dashed
index = np.argwhere(np.array([data.legendgroup for data in fig.data]) == 'uniform')[0][0]
fig.data[index].line = dict(color='red', width=2,
                             dash='dash')

# Layout
fig.layout['title'] = dict(text='Longitude of the ascending node distribution',
                        x=0.5)
fig.layout['xaxis_title'] = '$\Omega\:[deg]$'
fig.layout['yaxis_title'] = 'Kernel density estimation'
fig.layout['template'] = 'plotly_white'


fig.write_image("figures/oxp_dist.png", width=500, height=500, scale=2)
py.iplot(fig, filename="Longitude of the ascending node distribution")

# 4. Conclusion